A code stub for accessing the language dataset:


In [39]:
import numpy as np
from os.path import join
from google.colab import drive
import pickle

drive.mount('/content/drive/')

def load_pickle(path):
    with open(path, 'rb') as f:
        file = pickle.load(f)
        print ('Loaded %s..' %path)
        return file

dataset_directory = '/content/drive/My Drive/Major Project/'  ## CHANGE TO YOUR OWN DIRECTORY

emotions = ['anger', 'fear', 'joy', 'sadness']

tweets_train = np.load(join(dataset_directory, 'text_train_tweets.npy'))
labels_train = np.load(join(dataset_directory, 'text_train_labels.npy'))
vocabulary = load_pickle(join(dataset_directory, 'text_word_to_idx.pkl'))

tweets_val = np.load(join(dataset_directory, 'text_val_tweets.npy'))
labels_val = np.load(join(dataset_directory, 'text_val_labels.npy'))

tweets_test_public = np.load(join(dataset_directory, 'text_test_public_tweets_rand.npy'))

print(len(vocabulary))
idx_to_word = {i: w for w, i in vocabulary.items()}
for i in range(7):
  print(i, idx_to_word[i])

sample = 1  ## YOU CAN TRY OUT OTHER TWEETS

print('sample tweet, stored form:')
print(tweets_train[sample])
print(labels_train[sample])

print('sample tweet, readable form:')
decode = []
for i in range(50):
  decode.append(idx_to_word[tweets_train[sample][i]])
print(decode)
print(emotions[labels_train[sample]])


print(tweets_train.shape)
print(labels_train.shape)
print(tweets_val.shape)
print(labels_val.shape)
print(tweets_test_public.shape)




Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Loaded /content/drive/My Drive/Major Project/text_word_to_idx.pkl..
13978
0 <NULL>
1 <START>
2 <END>
3 it
4 makes
5 me
6 so
sample tweet, stored form:
[ 1 23 24 20 25 19 26 27 28  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
0
sample tweet, readable form:
['<START>', 'lol', 'adam', 'the', 'bull', 'with', 'his', 'fake', 'outrage', '<END>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>']
anger
(7098, 52)
(7098,)
(1460, 52)
(1460,

In [0]:
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,naive_bayes,svm
from sklearn.metrics import accuracy_score


In [0]:
np.random.seed(500)

##Preprocessing data

In [42]:
import numpy as np
from os.path import join
from google.colab import drive
import pickle
import pandas as pd
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection , naive_bayes , svm
from sklearn.metrics import accuracy_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def converttostring(raw_tweets):
    check ='<'
    tweets=[]
    for sample in range(len(raw_tweets)):
        decode=[]
        for i in range(len(raw_tweets[sample])):
            decode.append(idx_to_word[raw_tweets[sample][i]])
            res = [idx for idx in decode if idx[0].lower() !=check.lower()]
        a = " ".join(res)
        tweets.append(a)
    return tweets

In [0]:
tweets_train=converttostring(tweets_train)
tweets_test_public=converttostring(tweets_test_public)
tweets_val=converttostring(tweets_val)

In [0]:
tweetstrain=pd.DataFrame(tweets_train)
tweetstestpublic=pd.DataFrame(tweets_test_public)
tweetstrain=tweetstrain.rename(columns={0:'text'})
tweetstestpublic=tweetstestpublic.rename(columns={0:'text'})
tweetsval=pd.DataFrame(tweets_val)
tweetsval=tweetstrain.rename(columns={0:'text'})
tweetstrain['label']=pd.DataFrame(labels_train)
tweetsval['label']=pd.DataFrame(labels_val)

In [0]:
tweetstrain['text'].dropna(inplace =True)
tweetstestpublic['text'].dropna(inplace =True)
tweetsval['text'].dropna(inplace=True)
tweetstrain['text'] = [entry.lower() for entry in tweetstrain['text']]
tweetstestpublic['text'] = [entry.lower() for entry in tweetstestpublic['text']]
tweetsval['text']=[entry.lower() for entry in tweetsval['text'] ]
tweetstrain['text']= [word_tokenize (entry) for entry in tweetstrain['text']]
tweetstestpublic['text']= [word_tokenize (entry) for entry in tweetstestpublic['text']]
tweetsval['text']=[word_tokenize (entry) for entry in tweetsval['text']]


In [0]:
tagmap = defaultdict (lambda : wn.NOUN)
tagmap['J'] = wn.ADJ
tagmap['V'] = wn.VERB
tagmap['R'] = wn.ADV

In [48]:
for index, entry in enumerate(tweetstrain['text']):
  final_words = []
  word_lemmatized = WordNetLemmatizer() 
  for word, tag in pos_tag(entry):
    if word not in stopwords.words('english') and word.isalpha():
      word_final = word_lemmatized.lemmatize(word, tagmap[tag[0]]) 
      final_words.append(word_final)
  tweetstrain.loc[index, 'text_final'] = str(final_words)
print(tweetstrain['text_final'].head())

0    ['make', 'fuck', 'irate', 'jesus', 'nobody', '...
1           ['lol', 'adam', 'bull', 'fake', 'outrage']
2    ['pass', 'away', 'early', 'morning', 'fast', '...
3    ['lol', 'wow', 'gon', 'na', 'say', 'really', '...
4    ['need', 'sushi', 'date', 'olive', 'guarded', ...
Name: text_final, dtype: object


In [49]:
for index, entry in enumerate(tweetstestpublic['text']):
  final_words = []
  word_lemmatized = WordNetLemmatizer() 
  for word, tag in pos_tag(entry):
    if word not in stopwords.words('english') and word.isalpha():
      word_final = word_lemmatized.lemmatize(word, tagmap[tag[0]]) 
      final_words.append(word_final)
  tweetstestpublic.loc[index, 'text_final'] = str(final_words)
print(tweetstestpublic['text_final'].head())

0    ['omg', 'mother', 'daughter', 'dull', 'ni', 'm...
1    ['happy', 'birthday', 'miss', 'excited', 'back...
2    ['ever', 'cry', 'middle', 'bomb', 'rest', 'som...
3        ['mentally', 'suffered', 'worthless', 'pain']
4    ['courage', 'driver', 'shot', 'bus', 'show', '...
Name: text_final, dtype: object


In [51]:
for index, entry in enumerate(tweetsval['text']):
  final_words = []
  word_lemmatized = WordNetLemmatizer() 
  for word, tag in pos_tag(entry):
    if word not in stopwords.words('english') and word.isalpha():
      word_final = word_lemmatized.lemmatize(word, tagmap[tag[0]]) 
      final_words.append(word_final)
  tweetsval.loc[index, 'text_final'] = str(final_words)
print(tweetsval['text_final'].head())

0    ['make', 'fuck', 'irate', 'jesus', 'nobody', '...
1           ['lol', 'adam', 'bull', 'fake', 'outrage']
2    ['pass', 'away', 'early', 'morning', 'fast', '...
3    ['lol', 'wow', 'gon', 'na', 'say', 'really', '...
4    ['need', 'sushi', 'date', 'olive', 'guarded', ...
Name: text_final, dtype: object


In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(tweetstrain['text_final'],tweetstrain['label'], test_size=0.3)

In [0]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [54]:
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(tweetstrain['text_final'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(tweetstestpublic['text_final'])
X_val_tfidf = tfidf_vect.transform(tweetsval['text_final'])

##Implementing svm with linear kernel


In [0]:
from sklearn import model_selection, naive_bayes, svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

# Fit the training dataset.
SVM.fit(X_train_tfidf, y_train)

# Predict the labels on the validation dataset
SVMtest = SVM.predict(X_test_tfidf)
SVMtrain=SVM.predict(X_train_tfidf)

In [0]:
print("Accuracy scoore",accuracy_score(SVMtrain,y_train))

Accuracy scoore 0.964573268921095


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/Major Project/svmlinearafterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(SVMtest.shape[0]):

       writer.writerow([i+1,SVMtest[i]])

##Implementing SVM with polynomial rbf kernel

In [0]:
from sklearn import model_selection, naive_bayes, svm
rbfSVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')

# Fit the training dataset.
rbfSVM.fit(X_train_tfidf, y_train)

# Predict the labels on the validation dataset
rbfSVMtest = rbfSVM.predict(X_test_tfidf)
rbfSVMtrain=rbfSVM.predict(X_train_tfidf)

In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/svmrbfafterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(rbfSVMtest.shape[0]):

       writer.writerow([i+1,rbfSVMtest[i]])

In [0]:
from sklearn import model_selection, naive_bayes, svm

polySVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto')

# Fit the training dataset.
polySVM.fit(X_train_tfidf, y_train)

# Predict the labels on the validation dataset
polySVMtest = polySVM.predict(X_test_tfidf)
polySVMtrain=polySVM.predict(X_train_tfidf)


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/svmpolyafterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(polySVMtest.shape[0]):

       writer.writerow([i+1,polySVMtest[i]])

In [0]:
from sklearn import model_selection, naive_bayes, svm
SVM5 = svm.SVC(C=5.0, kernel='linear', degree=3, gamma='auto')

# Fit the training dataset.
SVM5.fit(X_train_tfidf, y_train)

# Predict the labels on the validation dataset
SVM5test = SVM5.predict(X_test_tfidf)
SVM5train=SVM5.predict(X_train_tfidf)

In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/svmlinear5afterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(SVM5test.shape[0]):

       writer.writerow([i+1,SVM5test[i]])

#Optimizing hyperparameters of random forest algorithm after preprocessing

In [0]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf2=RandomForestClassifier(random_state=123456)


clf2.fit(X_train_tfidf, y_train)
predictions_rand = clf2.predict(X_test_tfidf)
predictions_train = clf2.predict(X_train_tfidf)


In [0]:
from sklearn.model_selection import GridSearchCV

n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(clf2, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train_tfidf, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 52.8min finished


In [0]:
print(gridF.best_params_)

{'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 800}


In [0]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf2=RandomForestClassifier(random_state=123456,n_estimators=800,max_depth=30,min_samples_split=2,min_samples_leaf=1)


clf2.fit(X_train_tfidf, y_train)
predictions_rand = clf2.predict(X_test_tfidf)
predictions_train = clf2.predict(X_train_tfidf)

In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/randomforestoptimizedafterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(predictions_rand.shape[0]):

       writer.writerow([i+1,predictions_rand[i]])

#Implementing SVM with optimized hyperparameters

##Implementing neural network with keras

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
from keras.models import Sequential
from keras import layers
input_dim = X_train_tfidf.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(1000, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(500,activation='relu'))
model.add(layers.Dense(700,activation='relu'))
model.add(layers.Dense(800,activation='relu'))
model.add(layers.Dense(4, activation='softmax'))


In [62]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train_tfidf, y_train,epochs=20, batch_size=128, verbose=1,validation_data=(X_val_tfidf, tweetsval['label']))

Train on 4968 samples, validate on 7098 samples
Epoch 1/20
4968/4968 [==============================] - 2s 433us/step - loss: 1.0424 - accuracy: 0.5401 - val_loss: nan - val_accuracy: 0.0534
Epoch 2/20
4968/4968 [==============================] - 0s 89us/step - loss: 0.1863 - accuracy: 0.9461 - val_loss: nan - val_accuracy: 0.0545
Epoch 3/20
4968/4968 [==============================] - 0s 91us/step - loss: 0.0801 - accuracy: 0.9718 - val_loss: nan - val_accuracy: 0.0535
Epoch 4/20
4968/4968 [==============================] - 0s 89us/step - loss: 0.0602 - accuracy: 0.9740 - val_loss: nan - val_accuracy: 0.0544
Epoch 5/20
4968/4968 [==============================] - 0s 90us/step - loss: 0.0451 - accuracy: 0.9760 - val_loss: nan - val_accuracy: 0.0538
Epoch 6/20
4968/4968 [==============================] - 0s 88us/step - loss: 0.0400 - accuracy: 0.9773 - val_loss: nan - val_accuracy: 0.0538
Epoch 7/20
4968/4968 [==============================] - 0s 88us/step - loss: 0.0358 - accuracy: 0.9

In [0]:
g=model.predict_classes(X_test_tfidf)
h=model.predict_classes(X_train_tfidf)


In [0]:
print(h.shape)

(4968,)


In [0]:
print(h)

[0 1 0 ... 1 0 3]


In [0]:
print(y_train.shape)

(4968,)


In [0]:
print(y_train)

[0 1 0 ... 1 0 3]


In [64]:
print("accuracy score->",accuracy_score(h,y_train))

accuracy score-> 0.9786634460547504


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/Major Project/kerasprocessingdense.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(g.shape[0]):

       writer.writerow([i+1,g[i]])

##Keras with droput


In [0]:
from keras.models import Sequential
from keras import layers
input_dim = X_train_tfidf.shape[1]  # Number of features
modeldropout = Sequential()
modeldropout.add(layers.Dense(1000, input_dim=input_dim, activation='relu'))
modeldropout.add(layers.Dense(500,activation='relu'))
modeldropout.add(layers.Dropout(0.7))
modeldropout.add(layers.Dense(700,activation='relu'))
modeldropout.add(layers.Dense(800,activation='relu'))
modeldropout.add(layers.Dropout(0.8))
modeldropout.add(layers.Dense(4, activation='softmax'))

In [71]:
modeldropout.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
modeldropout.fit(X_train_tfidf, y_train,epochs=20, batch_size=128, verbose=1,validation_data=(X_val_tfidf, tweetsval['label']))

Train on 4968 samples, validate on 7098 samples
Epoch 1/20
4968/4968 [==============================] - 1s 129us/step - loss: 1.3435 - accuracy: 0.3277 - val_loss: nan - val_accuracy: 0.0551
Epoch 2/20
4968/4968 [==============================] - 0s 92us/step - loss: 0.7864 - accuracy: 0.6510 - val_loss: nan - val_accuracy: 0.0541
Epoch 3/20
4968/4968 [==============================] - 0s 91us/step - loss: 0.2564 - accuracy: 0.9259 - val_loss: nan - val_accuracy: 0.0526
Epoch 4/20
4968/4968 [==============================] - 0s 92us/step - loss: 0.1285 - accuracy: 0.9656 - val_loss: nan - val_accuracy: 0.0541
Epoch 5/20
4968/4968 [==============================] - 0s 91us/step - loss: 0.0839 - accuracy: 0.9748 - val_loss: nan - val_accuracy: 0.0544
Epoch 6/20
4968/4968 [==============================] - 0s 91us/step - loss: 0.0733 - accuracy: 0.9734 - val_loss: nan - val_accuracy: 0.0544
Epoch 7/20
4968/4968 [==============================] - 0s 90us/step - loss: 0.0608 - accuracy: 0.9

In [0]:
l=modeldropout.predict_classes(X_test_tfidf)
f=modeldropout.predict_classes(X_train_tfidf)

In [73]:
print("Accuracy score->",accuracy_score(f,y_train))

Accuracy score-> 0.9794685990338164


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/Major Project/kerasprocessingdropout.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(l.shape[0]):

       writer.writerow([i+1,l[i]])

##Neural network in keras with increased dropout and activation.

In [0]:
from keras.models import Sequential
from keras import layers
input_dim = X_train_tfidf.shape[1]  # Number of features
modeldropout = Sequential()
modeldropout.add(layers.Dense(1000, input_dim=input_dim, activation='relu'))
modeldropout.add(layers.Dense(500,activation='relu'))
modeldropout.add(layers.Dropout(0.9))
modeldropout.add(layers.Dense(700,activation='relu'))
modeldropout.add(layers.Activation('relu'))

modeldropout.add(layers.Dense(800,activation='relu'))
modeldropout.add(layers.Dropout(1.0))
modeldropout.add(layers.Dense(4, activation='softmax'))

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                  
model.fit(X_train_tfidf, y_train,epochs=20, batch_size=128, verbose=1,validation_data=(X_val_tfidf, tweets_val['label']))

Train on 4968 samples, validate on 7098 samples
Epoch 1/20
4968/4968 [==============================] - 1s 122us/step - loss: 1.3344 - accuracy: 0.3317 - val_loss: nan - val_accuracy: 0.0538
Epoch 2/20
4968/4968 [==============================] - 0s 100us/step - loss: 0.7075 - accuracy: 0.7872 - val_loss: nan - val_accuracy: 0.0540
Epoch 3/20
4968/4968 [==============================] - 1s 101us/step - loss: 0.1587 - accuracy: 0.9499 - val_loss: nan - val_accuracy: 0.0530
Epoch 4/20
4968/4968 [==============================] - 0s 98us/step - loss: 0.0800 - accuracy: 0.9706 - val_loss: nan - val_accuracy: 0.0537
Epoch 5/20
4968/4968 [==============================] - 0s 98us/step - loss: 0.0674 - accuracy: 0.9716 - val_loss: nan - val_accuracy: 0.0535
Epoch 6/20
4968/4968 [==============================] - 0s 95us/step - loss: 0.0605 - accuracy: 0.9728 - val_loss: nan - val_accuracy: 0.0537
Epoch 7/20
4968/4968 [==============================] - 0s 100us/step - loss: 0.0573 - accuracy: 

In [0]:
lincreasedropout=model.predict_classes(X_test_tfidf)
fincreasedropout=model.predict_classes(X_train_tfidf)

In [0]:
print("Accuracy score",accuracy_score(fincreasedropout,y_train))

Accuracy score 0.980072463768116


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/kerasprocessingincreasedropout.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(lincreasedropout.shape[0]):

       writer.writerow([i+1,lincreasedropout[i]])

##Implementing CNN

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb


In [0]:
max_features = 5000
maxlen = 52
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

In [0]:
print(tweets_train.shape)

(7098, 52)


In [0]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(4))
#model.add(Activation('sigmoid'))

Build model...


In [0]:
print(X_train_tfidf.shape)

(4968, 5000)


In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(tweets_train, labels_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(tweets_val,labels_val))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7098 samples, validate on 1460 samples
Epoch 1/10
7098/7098 [==============================] - 2s 247us/step - loss: 1.3706 - accuracy: 0.3478 - val_loss: 2.4378 - val_accuracy: 0.3103
Epoch 2/10
7098/7098 [==============================] - 1s 211us/step - loss: 1.1096 - accuracy: 0.5627 - val_loss: 2.9624 - val_accuracy: 0.3452
Epoch 3/10
7098/7098 [==============================] - 2s 213us/step - loss: 0.9105 - accuracy: 0.7037 - val_loss: 4.4912 - val_accuracy: 0.3808
Epoch 4/10
7098/7098 [==============================] - 2s 214us/step - loss: 0.7504 - accuracy: 0.7666 - val_loss: 5.9874 - val_accuracy: 0.3740
Epoch 5/10
7098/7098 [==============================] - 2s 212us/step - loss: 0.7402 - accuracy: 0.7282 - val_loss: 4.2788 - val_accuracy: 0.3281
Epoch 6/10
7098/7098 [==============================] - 2s 212us/step - loss: 0.8941 - accuracy: 0.6903 - val_loss: 1.4424 - val_accuracy: 0.3062
Epoch 7/10
7098/7098 [==============================] - 2s 213us/step - loss

In [0]:
gtrain=model.predict(tweets_train)
htest=model.predict(tweets_test_public)

In [0]:
print("accuracyscore",accuracy_score(gtrain,labels_train))

ValueError: ignored

In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/cnn.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(htest.shape[0]):

       writer.writerow([i+1,htest[i]])

#Implementing LSTM


In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D,SpatialDropout1D
from keras.datasets import imdb
from keras.callbacks import EarlyStopping

In [0]:
import tensorflow as tf

In [0]:
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [0]:
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 32

In [0]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_tfidf.shape[1]))
model.add(SpatialDropout1D(0.6))
model.add(LSTM(50, dropout=0.6, recurrent_dropout=0.6))
model.add(Dense(500,activation='relu'))
model.add(Dense(800,activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

history = model.fit(X_train_tfidf, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4471 samples, validate on 497 samples
Epoch 1/20
4471/4471 [==============================] - 1529s 342ms/step - loss: 1.3748 - accuracy: 0.3245 - val_loss: 1.3736 - val_accuracy: 0.3159
Epoch 2/20
4471/4471 [==============================] - 1549s 346ms/step - loss: 1.3749 - accuracy: 0.3245 - val_loss: 1.3749 - val_accuracy: 0.3159
Epoch 3/20
4471/4471 [==============================] - 1528s 342ms/step - loss: 1.3723 - accuracy: 0.3252 - val_loss: 1.3749 - val_accuracy: 0.3159
Epoch 4/20
4471/4471 [==============================] - 1480s 331ms/step - loss: 1.3727 - accuracy: 0.3252 - val_loss: 1.3742 - val_accuracy: 0.3159


In [0]:
g=model.predict_classes(X_train_tfidf)

In [0]:
print("Acccuracy score->",accuracy_score(g,y_train))

Acccuracy score-> 0.3242753623188406


##Optimized hyperparameters of svm


In [0]:
from sklearn.svm import SVC 


In [0]:
# train the model on train set 
model = SVC() 
model.fit(X_train_tfidf, y_train) 
  
# print prediction results 
predictions = model.predict(X_test_tfidf)

In [0]:
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','linear','poly']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train_tfidf, y_train) 

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.325, total=   2.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.323, total=   2.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.4s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.332, total=   2.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.325, total=   2.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.325, total=   2.7s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.411, total=   2.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.394, total=   2.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.419, total=   2.2s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.403, total=   2.1s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 375 out of 375 | elapsed: 14.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [0]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [0]:
grid_predictions = grid.predict(X_test_tfidf) 
gridpredictionstrain=grid.predict(X_train_tfidf)


In [0]:
print("Accuracy score->",accuracy_score(gridpredictionstrain,y_train))

Accuracy score-> 0.9776570048309179


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/svmhyperparamerterprocessing.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(grid_predictions.shape[0]):

       writer.writerow([i+1,grid_predictions[i]])

In [0]:

from sklearn.model_selection import RandomizedSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','linear','poly']} 
grid = RandomizedSearchCV(model, param_grid)
grid.fit(X_train_tfidf,y_train)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['rbf', 'linear', 'poly']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [0]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'kernel': 'rbf', 'gamma': 1, 'C': 10}
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [0]:
grid_predictions1 = grid.predict(X_test_tfidf) 
gridpredictionstrain1=grid.predict(X_train_tfidf)

In [0]:
print("Accuracy score:",accuracy_score(gridpredictionstrain1,y_train))

Accuracy score: 0.980475040257649


In [0]:
import csv

In [0]:
with open('/content/drive/My Drive/20comp8220/proj/text_dataset/svmhyperparamerterrandom.csv','w') as file:
     writer = csv.writer(file)
     writer.writerow(["ID","Prediction", ])
     for i in range(grid_predictions.shape[0]):

       writer.writerow([i+1,grid_predictions[i]])